#  Setup stuff: Connectivity

In [1]:
import os

from katana import remote


my_client = remote.Client()

print(my_client)


In [2]:
NUM_PARTITIONS  = 3
   #
DB_NAME         = "my_db"
GRAPH_NAME      = "my_graph"

print("--")


--


In [3]:
#  CONNECT TO GRAPH

for l_graph in my_client.get_database(name=DB_NAME).graphs_in_database():
   if (l_graph.name == GRAPH_NAME):
      my_graph=my_client.get_database(name=DB_NAME).get_graph_by_id(id=l_graph.graph_id)
         #
      break

print(my_graph)


<_Graph my_graph, Hfsb7j6DuhtaopxcqgJtLLu2Wmai4H2xLFijnftWC25b, 2>


#  Setup stuff: Debugging

In [ ]:
#  LIST ALL GRAPHS

for l_graph in my_client.graphs():
   print("GRAPH ID: ", l_graph.graph_id, "     DB GRAPH: ", l_graph.name, "    GRAPH VERSION: ", l_graph.version)

print("--")


In [ ]:
display(my_graph.num_nodes())
display(my_graph.num_edges())

In [ ]:
my_graph.query("CALL graph.schema() RETURN *")


In [ ]:
my_client.widgets().operations()


In [ ]:
# ##################################################################

In [4]:

l_result = my_graph.query("""

   MATCH (n) - [ r ] -> (m)
   RETURN n, m, r
   
   """, contextualize=True)

l_result.view()


          0/? [?op/s]

          0/? [?op/s]

#  Finally now, the Analytics ..

In [5]:
#  https://stackoverflow.com/questions/27600608/traverse-path-cypher-query

#  The full graph

l_result = my_graph.query("""

   MATCH (n: Airport ) - [ r ] -> (m: Airport)
   
   //  No data, there is no direct flight
   //
   // WHERE n.airport_code = "SJC" AND m.airport_code = "MKE"
   
   RETURN n, m, r
   
   """, contextualize=True)

l_result.view()


          0/? [?op/s]

          0/? [?op/s]

In [6]:
#  First traversal, Airport --> Airport, up to 3 in distance ..

l_result = my_graph.query("""

   MATCH (n: Airport )  - [r *..3] ->  (m: Airport)
   WHERE n.id = "SJC" AND m.id = "MKE"
   RETURN n.id, m.id
   
   """)

print(l_result[0:30])

#    m.airport_code n.airport_code
#  0            MKE            SJC
#  1            MKE            SJC

#  So Yes, answer, but no path ..


          0/? [?op/s]

  m.id n.id
0  MKE  SJC
1  MKE  SJC


In [7]:
#  Adding path ..

l_result = my_graph.query("""

   MATCH p= ( n: Airport )  - [r *..3] ->  (m: Airport)
   WHERE n.id = "SJC" AND m.id = "MKE"
   RETURN n.id AS FROM, m.id AS TO, p
   """)

print(l_result[0:30])

#  Currently, error ..
#
#  OperationError: query 9tWneccAey6jzpKLGGtto7v3X1iSkQDdRrSa7gcuv8fD-3qJysJrdkEo1NWz7V error: backtrace
#  (QueryClient.cpp:456): opgraph check failed (OpGraph.cpp:429): Unimplemented Features Pass
#  (OpGraphErrorChecker.cpp:12): backtrace (UnimplementedFeaturesChecker.cpp:360): backtrace
#  (OpGraph.cpp:682): backtrace (OpGraph.cpp:682): backtrace (OpGraph.cpp:682): backtrace
#  (OpGraph.cpp:682): backtrace (OpGraph.cpp:658): backtrace (UnimplementedFeaturesChecker.cpp:214):
#  accessing path variable p is not supported (UnimplementedFeaturesChecker.cpp:24): InternalError:NotImplemented


          0/? [?op/s]

OperationError: 5U16YeJjgDB78M8EVAgqTghHMr6zLUt3cGyipU6U8Tgs-5WAhiHvm3g5R5VU96 backtrace (QueryClient.cpp:605): opgraph check failed (OpGraph.cpp:464): Unimplemented Features Pass (OpGraphErrorChecker.cpp:12): backtrace (UnimplementedFeaturesChecker.cpp:376): backtrace (OpGraph.cpp:722): backtrace (OpGraph.cpp:722): backtrace (OpGraph.cpp:722): backtrace (OpGraph.cpp:698): backtrace (UnimplementedFeaturesChecker.cpp:231): accessing path variable p is not supported (UnimplementedFeaturesChecker.cpp:23): InternalError:NotImplemented: InternalError:NotImplemented

In [8]:
#  Using COLLECT to try and get path ..

l_result = my_graph.query("""

   MATCH ( n: Airport )  - [r *..3] ->  (m: Airport)
   WHERE n.id = "SJC" AND m.id = "MKE"
   
   RETURN n.id AS FROM, COLLECT(m.id)
   // RETURN COLLECT( [n.airport_code, m.airport_code] )                //  This did notthing effective
   
   """)

print(l_result[0:30])

#  This did not used to error. It used to return,
#
#    COLLECT(m.airport_code) FROM
#  0              [MKE, MKE]  SJC
# 
#  Not what we want

#  Now this gives,
#
#  OperationError: 9HaUVQpn6SQh83sR3oDGZLmtyzUoBgxEZeU26316cb4B-3Hc5zx68K6VVZyswB backtrace
#  (QueryClient.cpp:575): opgraph check failed (OpGraph.cpp:564): Unimplemented Features Pass 
#  (OpGraphErrorChecker.cpp:12): backtrace (UnimplementedFeaturesChecker.cpp:376): backtrace 
#  (OpGraph.cpp:822): backtrace (OpGraph.cpp:822): backtrace (OpGraph.cpp:798):
#  returning named variable paths not yet supported (UnimplementedFeaturesChecker.cpp:197):
#  InternalError:NotImplemented: InternalError:NotImplemented


          0/? [?op/s]

OperationError: FZc7WzfzBU1wRC2uSp7qphfTy3umAn89T5mMvLgQBCdQ-4UepibuFQQL9MLD3t backtrace (QueryClient.cpp:605): opgraph check failed (OpGraph.cpp:464): Unimplemented Features Pass (OpGraphErrorChecker.cpp:12): backtrace (UnimplementedFeaturesChecker.cpp:376): backtrace (OpGraph.cpp:722): backtrace (OpGraph.cpp:722): backtrace (OpGraph.cpp:698): returning named variable paths not yet supported (UnimplementedFeaturesChecker.cpp:197): InternalError:NotImplemented: InternalError:NotImplemented

In [9]:
#  Another attempt at using COLLECT, on the edge this time ..

l_result = my_graph.query("""

   MATCH ( n: Airport )  - [r *..3] ->  (m: Airport)
   WHERE n.id = "SJC" AND m.id = "MKE"
   RETURN n.id AS FROM, COLLECT(r.END_ID)
   """)

print(l_result[0:30])


#  OperationError: query 6jxiUgnCFao3yf2BTXPJqQDUTkAwTL4LMzmKjF6kNmD4-4BeDpVqisXe9Ha8QW error:
#  backtrace (QueryClient.cpp:452): backtrace (OpGraph.cpp:313): backtrace (OpGraph.h:588):
#  backtrace (OpGraph.h:588): backtrace (OpGraph.h:588): backtrace (OpGraph.h:588): backtrace
#  (OpGraph.cpp:320): backtrace (RewritePattern.h:46): backtrace (MatchOrderOptimizer.cpp:102):
#  backtrace (MatchOperator.cpp:2062): returning named variable paths not yet supported
#  (MatchOperator.cpp:1894): InternalError:NotImplemented


          0/? [?op/s]

OperationError: FeAsAF1eEE4WY4sSju4pYhukaKqZ7V344zUgQpSw9URb-3eXgu8kw2C4pTKRiX backtrace (QueryClient.cpp:605): opgraph check failed (OpGraph.cpp:464): Unimplemented Features Pass (OpGraphErrorChecker.cpp:12): backtrace (UnimplementedFeaturesChecker.cpp:376): backtrace (OpGraph.cpp:722): backtrace (OpGraph.cpp:722): backtrace (OpGraph.cpp:698): returning named variable paths not yet supported (UnimplementedFeaturesChecker.cpp:197): InternalError:NotImplemented: InternalError:NotImplemented

In [ ]:
# ##################################################################

In [ ]:
#  Second traversal, Airport --> Airport, up to 3 in distance, but in other direction ..

l_result = my_graph.query("""

   MATCH (n: Airport )  - [r: FLIES_TO *..3] ->  (m: Airport)
   WHERE n.id = "MKE" AND m.id = "SJC"
   RETURN n.id, m.id
   
   """)

print(l_result[0:30])


In [ ]:
# ##################################################################

In [10]:

#  From Roshan and Boris ..

l_result = my_graph.query("""

   MATCH ( n: Airport )  - [] ->  (m: Airport)
   WHERE n.id = "SJC" AND m.id = "MKE"
   RETURN "1" AS num_hops, n.id AS FROM, [m.id] as path
      
   UNION
   
   MATCH ( n: Airport )-[]->(p1:Airport)-[]-> (m: Airport)
   WHERE n.id = "SJC" AND m.id = "MKE"
   RETURN "2" AS num_hops, n.id AS FROM, [p1.id,m.id] as path
      
   UNION
   
   MATCH ( n: Airport )-[]->(p1:Airport)-[]->(p2:Airport)-[]-> (m: Airport)
   WHERE n.id = "SJC" AND m.id = "MKE"
   RETURN "3" AS num_hops, n.id AS FROM, [p1.id,p2.id,m.id] as path
   
   """)

print(l_result[0:30])
    
#  Outputs,
#
#    FROM num_hops             path
#  0  SJC        3  [DEN, ORD, MKE]
#  1  SJC        2       [ORD, MKE]



          0/? [?op/s]

  FROM num_hops             path
0  SJC        3  [DEN, ORD, MKE]
1  SJC        2       [ORD, MKE]
